### 1. Instalación de las Librerías


In [1]:
! pip install openai azure-search-documents python-dotenv pandas numpy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### 2. Cargar Variables de Entorno

In [10]:
from dotenv import load_dotenv, find_dotenv
import os

# Cargar las variables de entorno
load_dotenv(find_dotenv())

# Variables necesarias
API_KEY = os.getenv("OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE", "").strip()
API_TYPE = os.getenv("OPENAI_API_TYPE", "azure")
API_VERSION = os.getenv("OPENAI_API_VERSION", "2023-05-15")
CHAT_MODEL = os.getenv("CHAT_MODEL_NAME")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL_NAME")
JSON_FILES_PATH = os.getenv("JSON_FILES_PATH")

# Verificar que el archivo existe
if not os.path.isfile(JSON_FILES_PATH):
    raise FileNotFoundError(f"El archivo {JSON_FILES_PATH} no existe.")

print(f"Configuración cargada:\n- Archivo JSON: {JSON_FILES_PATH}\n- Modelo de chat: {CHAT_MODEL}\n- Modelo de embeddings: {EMBEDDING_MODEL}")


Configuración cargada:
- Archivo JSON: /Users/luisalbertocerelli/Desktop/00-Todo/14_Hackaton_Real/PRINCIPAL/01Conversion_File_Json/Json_conversion/documentos_indexados.json
- Modelo de chat: gpt-35-turbo
- Modelo de embeddings: text-embedding-ada-002


### 3. Configuración de OpenAI con Azure


In [12]:
import openai

# Configuracion de OpenAI:

openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = "2023-06-01-preview"

print("Configuración de OpenAI completada.")

Configuración de OpenAI completada.


### 4. Configuración de Azure Cognitive Search

Es el servicio para almacenar y buscar documentos relevantes. Aquí configuramos:

* El endpoint de tu servicio de búsqueda.
* La clave API para autenticación.
* El cliente para interactuar con el índice.

In [14]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv, find_dotenv
import os

# Cargar las variables de entorno
load_dotenv(find_dotenv())

# Configuración de Azure Cognitive Search desde el .env
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("SEARCH_INDEX_NAME")
search_api_key = os.getenv("SEARCH_API_KEY")

# Validar que las variables necesarias están configuradas
if not search_service_endpoint or not index_name or not search_api_key:
    raise ValueError("Faltan valores en el archivo .env para configurar Azure Cognitive Search.")

# Crear el cliente de búsqueda
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

print("Cliente de búsqueda configurado correctamente.")


Cliente de búsqueda configurado correctamente.


### 5. Función para Cargar Documentos en Azure Cognitive Search

In [19]:
import os
import json
import base64
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchableField, SimpleField, SearchIndex
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import re

# Cargar variables del archivo .env
load_dotenv()
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("SEARCH_API_KEY")
index_name = os.getenv("INDEX_NAME")
json_files_path = os.getenv("JSON_FILES_PATH")

def normalizar_clave(clave):
    """
    Normaliza la clave para cumplir con los requisitos de Azure Search.
    Convierte caracteres no válidos en guiones bajos (_).
    """
    # Reemplazar caracteres no permitidos con guión bajo
    return re.sub(r"[^a-zA-Z0-9_\-]", "_", clave)

def cargar_documentos_en_search():
    # Validar variables esenciales
    if not search_service_endpoint or not search_api_key or not index_name:
        print("Error: Asegúrate de que las variables SEARCH_SERVICE_ENDPOINT, SEARCH_API_KEY y INDEX_NAME están definidas en el archivo .env")
        return

    # Crear cliente para el índice
    index_client = SearchIndexClient(
        endpoint=search_service_endpoint,
        credential=AzureKeyCredential(search_api_key)
    )

    # Definir el esquema del índice
    fields = [
        SimpleField(name="id", type="Edm.String", key=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="en.microsoft"),
    ]
    index = SearchIndex(name=index_name, fields=fields)

    # Crear el índice si no existe
    try:
        if index_name not in [i.name for i in index_client.list_indexes()]:
            index_client.create_index(index)
            print(f"Índice '{index_name}' creado exitosamente.")
        else:
            print(f"Índice '{index_name}' ya existe. Continuando...")
    except Exception as e:
        print(f"Error al gestionar el índice: {e}")
        return

    # Cargar documentos desde el archivo JSON
    try:
        with open(json_files_path, "r", encoding="utf-8") as json_file:
            raw_documents = json.load(json_file)
            # Normalizar las claves de los documentos
            documents = [
                {"id": normalizar_clave(doc["id"]), "content": doc["content"]}
                for doc in raw_documents
                if "id" in doc and "content" in doc
            ]
    except Exception as e:
        print(f"Error al leer el archivo JSON: {e}")
        return

    # Crear cliente para interactuar con el índice
    search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name=index_name,
        credential=AzureKeyCredential(search_api_key)
    )

    # Subir documentos al índice
    try:
        upload_response = search_client.upload_documents(documents=documents)
        print(f"Documentos cargados exitosamente: {upload_response}")
    except Exception as e:
        print(f"Error al cargar documentos en el índice: {e}")

# Llamar a la función
if __name__ == "__main__":
    cargar_documentos_en_search()


Índice 'documentos-huella' ya existe. Continuando...
Documentos cargados exitosamente: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x160693da0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aa2d0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606a9880>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aaf00>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aad50>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aae40>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aacc0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aad20>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aae70>, <azure.search.documents._generated.models._models_py3.IndexingResult object

### 6. Búsqueda de Documentos Relevantes (solo para probar si esta funcionando bien)

In [28]:
import os
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Configuración desde las variables del entorno
service_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
api_key = os.environ["SEARCH_API_KEY"]
index_name = os.environ["INDEX_NAME"]

# Crear cliente de búsqueda
search_client = SearchClient(
    endpoint=service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(api_key)
)

# Función para buscar documentos
def buscar_documentos(query, top=5):
    results = search_client.search(
        search_text=query,
        top=top,
        query_type="simple"
    )
    documentos = [doc['content'] for doc in results]
    return documentos

# Ejemplo de uso
if __name__ == "__main__":
    query = "huella de carbono"
    resultados = buscar_documentos(query, top=3)
    print("Resultados de la búsqueda:")
    for i, doc in enumerate(resultados, start=1):
        print(f"{i}. {doc}")



Resultados de la búsqueda:
1. Hola y bienvenidos al Cuestionario de Gestión de proyectos de Huella 
de Carbono 
Gracias por leer, completar los datos requeridos y enviarlo
Hemos diseñado este cuestionario para poder comprender mejor la situación de su entidad en términos
de sostenibilidad, su disposición a embarcarse en un plan de gestión de huella de carbono con el soporte
de la Asociación Huella de Carbono y así poder ayudarle mejor y de forma más personalizada.
Su participación en este cuestionario es esencial para obtener una visión completa y precisa de la
información que necesitamos para poder ayudarle.
Este cuestionario se ha diseñado para ser claro y fácil de seguir.
Los campos con un asterisco en Rojo * son obligatorios
Las preguntas abiertas están destinadas a recoger sus valiosas opiniones y experiencias, mientras que
las preguntas cerradas nos proporcionarán datos cuantitativos que serán fundamentales para nuestro
análisis.
Las respuestas serán utilizadas para preparar el p

### Comienzo con el orquestador LangChain

Importamos las libreria: 

In [34]:
! pip install langchain azure-search-documents python-dotenv matplotlib scikit-learn
! pip install --upgrade langchain langchain-community azure-search-documents openai python-dotenv matplotlib



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 13.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.4
    Uninstalling matplotlib-3.8.4:
      Successfully uninstalled matplotlib-3.8.4
  Attempting uninstall: openai
    Found existing installation: openai 0.27.0
    Uninstalling openai-0.27.0:
      Successfully uninstalled openai-0.27.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### Generamos codigo de pregunta y validamos la respuesta: 

#### 1-Configuramos las variables de entorno: 

In [35]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configuración
SEARCH_SERVICE_ENDPOINT = os.environ["SEARCH_SERVICE_ENDPOINT"]
SEARCH_API_KEY = os.environ["SEARCH_API_KEY"]
INDEX_NAME = os.environ["INDEX_NAME"]

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]  # Modelo: text-embedding-3-small


#### Creamos el Cliente Azure Search

In [36]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Crear cliente de Azure Search
search_client = SearchClient(
    endpoint=SEARCH_SERVICE_ENDPOINT,
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(SEARCH_API_KEY)
)


#### Configuramos Azure OpenAI para Embeddings y Respuestas

In [43]:
import openai

# Configurar cliente de OpenAI para Azure
openai.api_type = "azure"
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_version = "2023-05-15"
openai.api_key = os.environ["OPENAI_API_KEY"]

# Función para generar embeddings usando Azure OpenAI
def generar_embeddings(texto):
    response = openai.Embedding.create(
        input=texto,
        model=os.environ["EMBEDDING_MODEL_NAME"]  # text-embedding-3-small
    )
    return response['data'][0]['embedding']




#### Funcion para responder Prompts

In [44]:
def generar_respuesta(prompt):
    response = openai.ChatCompletion.create(
        engine="gpt-4",  # Reemplaza con el nombre de tu deployment en Azure
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']


####  Configuración del Orquestador LangChain

Instalacion de librerias: 

In [ ]:
! pip install --upgrade langchain openai azure-search-documents langchain-openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [87]:
! pip install --upgrade openai langchain azure-search-documents langchain-openai




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


#### Crear una solucion personalizada para Embeddings

In [92]:
import openai
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

# Configurar OpenAI para Azure
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE").rstrip("/")  # Ajustar endpoint
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Nombre del deployment
embedding_deployment = "text-embedding-ada-002"  # Usa el nombre correcto del deployment

# Función para generar embeddings
def generar_embeddings(texto):
    try:
        response = openai.Embedding.create(
            input=texto,
            engine=embedding_deployment  # Usa el deployment válido
        )
        return response['data'][0]['embedding']
    except Exception as e:
        print(f"Error al generar embeddings: {e}")
        return [0.0] * 1536  # Tamaño del vector para `text-embedding-ada-002`


#### Integrar con LangChain

#### Configuración Completa de AzureSearch

In [93]:
from langchain.vectorstores import AzureSearch

vectorstore = AzureSearch(
    azure_search_endpoint=os.getenv("SEARCH_SERVICE_ENDPOINT"),
    azure_search_key=os.getenv("SEARCH_API_KEY"),
    index_name=os.getenv("INDEX_NAME"),
    embedding_function=generar_embeddings  # Usa la función personalizada corregida
)


Error al generar embeddings: No module named 'openai.object_classes'


####  Prueba de Flujo Completo

In [84]:
def main():
    prompt = "¿Qué es la huella de carbono?"
    try:
        print(f"Generando embeddings para: {prompt}")
        embedding = generar_embeddings(prompt)
        print(f"Embedding generado: {embedding[:5]}... (truncado)")

        # Prueba del vectorstore
        print("Probando AzureSearch...")
        docs = vectorstore.similarity_search(prompt, k=3)
        print(f"Documentos encontrados: {docs}")

    except Exception as e:
        print(f"Error en el flujo: {e}")

if __name__ == "__main__":
    main()


Generando embeddings para: ¿Qué es la huella de carbono?
Error al generar embeddings: No module named 'openai.object_classes'
Embedding generado: [0.0, 0.0, 0.0, 0.0, 0.0]... (truncado)
Probando AzureSearch...
Error al generar embeddings: No module named 'openai.object_classes'
Error en el flujo: (InvalidRequestParameter) Unknown field 'content_vector' in vector field list.
Code: InvalidRequestParameter
Message: Unknown field 'content_vector' in vector field list.
Exception Details:	(UnknownField) Unknown field 'content_vector' in vector field list.
	Code: UnknownField
	Message: Unknown field 'content_vector' in vector field list.


#### Validacion de respuestas: 

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

def validar_respuesta(prompt):
    # Generar respuesta de la IA
    respuesta_ia = qa_chain.run(prompt)
    print(f"Respuesta de la IA: {respuesta_ia}")
    
    # Recuperar documentos relevantes
    documentos = vectorstore.similarity_search(prompt, k=3)
    embeddings_docs = [generar_embeddings(doc.page_content) for doc in documentos]
    embedding_ia = generar_embeddings(respuesta_ia)

    # Calcular similitud
    similitudes = [cosine_similarity([embedding_ia], [doc_emb])[0][0] for doc_emb in embeddings_docs]
    max_similitud = max(similitudes)

    return respuesta_ia, max_similitud, documentos


#### Ejemplo completo de flujo

In [50]:
def main():
    prompt = "¿Qué es la huella de carbono?"
    print(f"Prompt: {prompt}")

    # Validar respuesta
    respuesta, similitud, docs = validar_respuesta(prompt)
    print(f"Similitud máxima: {similitud:.2f}")
    print("Documentos relevantes:")
    for doc in docs:
        print(f"- {doc.page_content}")

if __name__ == "__main__":
    main()


Prompt: ¿Qué es la huella de carbono?


NameError: name 'qa_chain' is not defined

#### Funcion para buscar documentos en Azure AI Search

In [39]:
def buscar_documentos(query, top=3):
    results = search_client.search(search_text=query, top=top)
    documentos = [doc['content'] for doc in results]
    return documentos


#### Generamos la respuesta y la validamos: 

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Función para validar respuestas
def validar_respuesta(prompt, respuesta_ia):
    # Buscar documentos relevantes
    documentos = buscar_documentos(prompt)
    embeddings_docs = [generar_embeddings(doc) for doc in documentos]
    embedding_ia = generar_embeddings(respuesta_ia)

    # Calcular similitudes
    similitudes = [cosine_similarity([embedding_ia], [doc_emb])[0][0] for doc_emb in embeddings_docs]
    max_similitud = max(similitudes)
    
    return max_similitud, documentos


#### Ejemplo complejo de flujo: 

In [45]:
# Función principal
def main():
    prompt = "¿Qué es la huella de carbono?"
    print(f"Prompt: {prompt}")

    # Generar respuesta de la IA
    respuesta_ia = openai_client.chat.completions.create(
        model="gpt-4",  # Modelo configurado
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content

    print(f"Respuesta de la IA: {respuesta_ia}")

    # Validar respuesta
    similitud, docs = validar_respuesta(prompt, respuesta_ia)
    print(f"Similitud máxima con documentos: {similitud:.2f}")
    print("Documentos relevantes:")
    for doc in docs:
        print(f"- {doc}")

if __name__ == "__main__":
    main()


Prompt: ¿Qué es la huella de carbono?


NameError: name 'openai_client' is not defined

In [46]:
import os
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores.azuresearch import AzureSearch
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Cargar variables de entorno
load_dotenv()

# Configuración Azure
search_service_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
search_api_key = os.environ["SEARCH_API_KEY"]
index_name = os.environ["INDEX_NAME"]
openai_api_key = os.environ["OPENAI_API_KEY"]
openai_api_base = os.environ["OPENAI_API_BASE"]
openai_model_name = os.environ["CHAT_MODEL_NAME"]

# Conectar a Azure Search
vectorstore = AzureSearch(
    azure_search_endpoint=search_service_endpoint,
    azure_search_key=search_api_key,
    index_name=index_name,
    embedding_function=None,  # Configura embeddings si las usas
)

# Configurar el modelo de Azure OpenAI
llm = AzureOpenAI(
    deployment_name=openai_model_name,
    model="gpt-4",
    azure_endpoint=openai_api_base,
    api_key=openai_api_key
)

# Crear el flujo de preguntas y respuestas
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

# Función para ejecutar y validar respuestas
def ejecutar_y_validar(prompt, respuesta_correcta):
    # Generar respuesta con la IA
    respuesta_ia = qa_chain.run(prompt)
    print(f"Pregunta: {prompt}")
    print(f"Respuesta de la IA: {respuesta_ia}")
    print(f"Respuesta Correcta: {respuesta_correcta}")
    
    # Validar respuesta
    es_correcta = respuesta_correcta.lower() in respuesta_ia.lower()
    return es_correcta

# Métricas y Visualización
def mostrar_metricas(resultados):
    preguntas, correctas = zip(*resultados)
    accuracy = accuracy_score([1] * len(correctas), correctas)
    
    # Visualizar
    plt.bar(preguntas, correctas, color='blue')
    plt.title(f"Accuracy: {accuracy * 100:.2f}%")
    plt.xlabel("Preguntas")
    plt.ylabel("Respuestas Correctas (1=Sí, 0=No)")
    plt.xticks(rotation=45)
    plt.show()

# Ejemplo de Prompts y Respuestas
prompts_respuestas = [
    {"prompt": "¿Qué es la huella de carbono?", "correcta": "La huella de carbono se mide en toneladas de CO2."},
    {"prompt": "¿Cómo afecta el cambio climático?", "correcta": "Afecta a todos los ecosistemas de la Tierra."},
]

# Ejecutar el flujo
resultados = []
for item in prompts_respuestas:
    es_correcta = ejecutar_y_validar(item["prompt"], item["correcta"])
    resultados.append((item["prompt"], int(es_correcta)))

# Mostrar métricas
mostrar_metricas(resultados)


TypeError: 'NoneType' object is not callable

### Uso:

In [29]:
query = "¿Qué es la huella de carbono?"
documentos = buscar_documentos(query)
print("Documentos relevantes:", documentos)


Documentos relevantes: ['Hola y bienvenidos al Cuestionario de Gestión de proyectos de Huella \nde Carbono \nGracias por leer, completar los datos requeridos y enviarlo\nHemos diseñado este cuestionario para poder comprender mejor la situación de su entidad en términos\nde sostenibilidad, su disposición a embarcarse en un plan de gestión de huella de carbono con el soporte\nde la Asociación Huella de Carbono y así poder ayudarle mejor y de forma más personalizada.\nSu participación en este cuestionario es esencial para obtener una visión completa y precisa de\xa0la\ninformación que necesitamos para poder ayudarle.\nEste cuestionario se ha diseñado para ser claro y fácil de seguir.\nLos campos con un asterisco en Rojo * son obligatorios\nLas preguntas abiertas están destinadas a recoger sus valiosas opiniones y experiencias, mientras que\nlas preguntas cerradas nos proporcionarán datos cuantitativos que serán fundamentales para nuestro\nanálisis.\nLas respuestas serán\xa0utilizadas para

### 7. Validación de Respuestas

In [30]:
def validar_respuesta(query, respuesta_ia):
    documentos_relevantes = buscar_documentos(query)
    interseccion = sum([1 for doc in documentos_relevantes if doc in respuesta_ia])
    precision = interseccion / len(documentos_relevantes) if documentos_relevantes else 0
    recall = interseccion / len(respuesta_ia.split()) if respuesta_ia else 0
    return {"precision": precision, "recall": recall, "docs_relevantes": documentos_relevantes}


### 8. Interacción con Azure OpenAI
Se genera una respuesta basada en un modelo de chat y se valida.

In [47]:
query = "¿Cómo se mide la huella de carbono?"
respuesta_ia = openai.ChatCompletion.create(
    engine=CHAT_MODEL,
    messages=[
        {"role": "system", "content": "Eres un experto en sostenibilidad."},
        {"role": "user", "content": query}
    ],
    max_tokens=100
)["choices"][0]["message"]["content"]

# Validación
metrics = validar_respuesta(query, respuesta_ia)
print("Métricas de validación:", metrics)


InvalidRequestError: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.